<a href="https://colab.research.google.com/github/parsakzr/codellm/blob/master/llm_gguf_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q huggingface-hub
!git clone https://github.com/ggerganov/llama.cpp.git
!pip install -q -r llama.cpp/requirements.txt


Cloning into 'llama.cpp'...
remote: Enumerating objects: 16244, done.
remote: Counting objects: 100% (3226/3226), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 16244 (delta 3129), reused 3125 (delta 3071), pack-reused 13018
Receiving objects: 100% (16244/16244), 18.62 MiB | 17.02 MiB/s, done.
Resolving deltas: 100% (11304/11304), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB

In [2]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [21]:
%cd llama.cpp
!mkdir build
%cd build
!cmake ..
!cmake --build . --config Release
%cd ../..

/content/llama.cpp
/content/llama.cpp/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1") 
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- x86 detected
-- Configuring done (1.3s)
-- Generating done (0.1s)
-- Build files have been written to: /content/llama.cpp/build
[  0%] Building C object CMakeFiles/ggml.dir/ggml.c.o
[  1%] Building C object CMakeFiles/ggml.dir/ggml-alloc.c.

In [25]:
!ls llama.cpp/build

bin		     common		    examples	       LlamaConfigVersion.cmake  tests
CMakeCache.txt	     compile_commands.json  libggml_static.a   Makefile
CMakeFiles	     CTestTestfile.cmake    libllama.a	       pocs
cmake_install.cmake  DartConfiguration.tcl  LlamaConfig.cmake  Testing


In [42]:
from huggingface_hub import snapshot_download

model_id = "parsak/phi-2-code-instruct"
local_model = model_id.split("/")[-1]

snapshot_download(
        repo_id=model_id,
        local_dir=local_model,
        local_dir_use_symlinks=False,
        revision="main",
)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.37k [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

'/content/phi-2-code-instruct'

In [43]:
!python llama.cpp/convert.py -h

/content/llama.cpp/gguf-py
usage: convert.py [-h] [--awq-path AWQ_PATH] [--dump] [--dump-single] [--vocab-only]
                  [--outtype {f32,f16,q8_0}] [--vocab-dir VOCAB_DIR] [--vocab-type {spm,bpe,hfft}]
                  [--pad-vocab] [--outfile OUTFILE] [--ctx CTX] [--concurrency CONCURRENCY]
                  [--big-endian]
                  model

Convert a LLaMa model to a GGML compatible file

positional arguments:
  model                 Directory containing the model file or the model file itself (*.pth, *.pt,
                        *.bin)

options:
  -h, --help            show this help message and exit
  --awq-path AWQ_PATH   Path to the Activation-aware Weight Quantization cache file
  --dump                Display the model content without converting it
  --dump-single         Display the content of a single model file without conversion
  --vocab-only          Extract and output only the vocabulary
  --outtype {f32,f16,q8_0}
                        Output format - 

In [44]:
!python llama.cpp/convert-hf-to-gguf.py -h

usage: convert-hf-to-gguf.py [-h] [--vocab-only] [--awq-path AWQ_PATH] [--outfile OUTFILE]
                             [--outtype {f32,f16}] [--bigendian]
                             model

Convert a huggingface model to a GGML compatible file

positional arguments:
  model                directory containing model file

options:
  -h, --help           show this help message and exit
  --vocab-only         extract only the vocab
  --awq-path AWQ_PATH  Path to scale awq cache file
  --outfile OUTFILE    path to write to; default: based on input
  --outtype {f32,f16}  output format - use f32 for float32, f16 for float16
  --bigendian          model is executed on big endian machine


In [45]:
!ls -la $local_model

total 5432508
drwxr-xr-x 2 root root       4096 Jan 18 12:35 .
drwxr-xr-x 1 root root       4096 Jan 18 12:35 ..
-rw-r--r-- 1 root root       1080 Jan 18 12:30 added_tokens.json
-rw-r--r-- 1 root root        900 Jan 18 12:30 config.json
-rw-r--r-- 1 root root         74 Jan 18 12:30 generation_config.json
-rw-r--r-- 1 root root       1519 Jan 18 12:30 .gitattributes
-rw-r--r-- 1 root root     456318 Jan 18 12:30 merges.txt
-rw-r--r-- 1 root root 4995584424 Jan 18 12:35 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root  563832976 Jan 18 12:31 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root      35716 Jan 18 12:30 model.safetensors.index.json
-rw-r--r-- 1 root root       5393 Jan 18 12:30 README.md
-rw-r--r-- 1 root root        473 Jan 18 12:30 special_tokens_map.json
-rw-r--r-- 1 root root       7371 Jan 18 12:30 tokenizer_config.json
-rw-r--r-- 1 root root    2114924 Jan 18 12:30 tokenizer.json
-rw-r--r-- 1 root root     798156 Jan 18 12:30 vocab.json


In [53]:
!python llama.cpp/convert-hf-to-gguf.py --outfile $local_model.gguf --outtype f16 phi-2-code-instruct

Loading model: phi-2-code-instruct
gguf: This GGUF file is for Little Endian only
Set model parameters
Set model tokenizer
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
gguf: Adding 50000 merge(s).
gguf: Setting special token type bos to 50256
gguf: Setting special token type eos to 50256
gguf: Setting special token type unk to 50256
gguf: Setting special token type pad to 50256
Exporting model to '.gguf'
gguf: loading model part 'model-00001-of-00002.safetensors'
token_embd.weight, n_dims = 2, torch.float16 --> float16
blk.0.attn_norm.bias, n_dims = 1, torch.float16 --> float32
blk.0.attn_norm.weight, n_dims = 1, torch.float16 --> float32
blk.0.ffn_up.bias, n_dims = 1, torch.float16 --> float32
blk.0.ffn_up.weight, n_dims = 2, torch.float16 --> float16
blk.0.ffn_down.bias, n_dims = 1, torch.float16 --> float32
blk.0.ffn_down.weight, n_dims = 2, torch.float16 --> float16
blk.0.attn_output.bias, n_dims = 1, torch.fl

In [59]:
# quantization
qmodels_dir = local_model + '-gguf'
!mkdir $qmodels_dir

quantizations = ['Q4_0', "Q4_K_M", "Q3_K_M"] #, 'q8_0', 'q16_0']

mkdir: cannot create directory ‘phi-2-code-instruct-gguf’: File exists
phi-2-code-instruct.gguf


In [58]:
!mv .gguf phi-2-code-instruct-gguf/phi-2-code-instruct.gguf

In [28]:
for quantization in quantizations:
    !llama.cpp/build/bin/quantize $qmodels_dir/$local_model-f16.gguf $qmodels_dir/$local_model-$quantization.gguf $quantization

main: missing ftype


In [61]:
!llama.cpp/build/bin/quantize /content/phi-2-code-instruct-gguf/phi-2-code-instruct-f16.gguf /content/phi-2-code-instruct-gguf/phi-2-code-instruct-q4_k_m.gguf Q4_K_M

main: build = 1906 (dcad445)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/phi-2-code-instruct-gguf/phi-2-code-instruct-f16.gguf' to '/content/phi-2-code-instruct-gguf/phi-2-code-instruct-q4_k_m.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 20 key-value pairs and 453 tensors from /content/phi-2-code-instruct-gguf/phi-2-code-instruct-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi2
llama_model_loader: - kv   1:                               general.name str              = Phi2
llama_model_loader: - kv   2:                        phi2.context_length u32              = 2048
llama_model_loader: - kv   3:                      phi2.embedding_length u32              = 2560
llama_model_loader: - kv   4:                   phi2.feed_forward_lengt

In [62]:
from huggingface_hub import HfApi
api = HfApi()

model_id_gguf = model_id + "-gguf"
api.create_repo(model_id_gguf, exist_ok=True, repo_type="model")
api.upload_folder(
    folder_path=qmodels_dir,
    repo_id=model_id_gguf,
    repo_type="model",
    commit_message="upload quantizations",
    # private=False,
)
# api.upload_file(
#     path_or_fileobj="/content/mistral-code-7b-instruct-gguf/mistral-code-7b-instruct-q6_k.gguf",
#     path_in_repo="mistral-code-7b-instruct-q6_k.gguf",
#     repo_id=model_id_gguf,
#     # commit_message="upload quantizations",
# )

phi-2-code-instruct-f16.gguf:   0%|          | 0.00/5.56G [00:00<?, ?B/s]

phi-2-code-instruct-q4_0.gguf:   0%|          | 0.00/1.60G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

phi-2-code-instruct-q4_k_m.gguf:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/parsak/phi-2-code-instruct-gguf/commit/22d3423e9990821f756bef8722501d873325b487', commit_message='upload quantizations', commit_description='', oid='22d3423e9990821f756bef8722501d873325b487', pr_url=None, pr_revision=None, pr_num=None)